# Classifier Trainig

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix, ConfusionMatrixDisplay
import scipy.stats as stats
from tqdm import tqdm
from tqdm.keras import TqdmCallback
from datetime import datetime
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Input, Dense, GRU, GRUCell, LSTM, LSTMCell, RNN, SimpleRNN

from MotionClassifier import MotionClassifier

import sys, os

c:\Users\itaym\Documents\limudim\semester_8\robotic_hand_project\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NotFoundError: c:\Users\itaym\Documents\limudim\semester_8\robotic_hand_project\.venv\lib\site-packages\tensorflow_decision_forests\tensorflow\ops\inference\inference.so not found

In [ ]:
N_EPOCHS = 500
# N_FEATURES = 6 # accel.x, accel.y, accel.z, gyro.x, gyro.y, gyro.z 
N_FEATURES = 3 # accel.x, accel.y, accel.z
SAMPLE_LEN = 400

In [ ]:
tf.config.list_physical_devices()

In [ ]:
tf.config.list_physical_devices('GPU')
if len(tf.config.list_physical_devices('GPU')):
    print("GPU found, hurray =]")

## Read Data

In [ ]:

samples= {
    'Rest':    [],
    'Up':      [],
    'Down':    [],
    'Forward': [],
    'Back':    []
}

label_to_OH = {
    'Rest':    (1., 0., 0., 0., 0.),
    'Up':      (0., 1., 0., 0., 0.),
    'Down':    (0., 0., 1., 0., 0.),
    'Forward': (0., 0., 0., 1., 0.),
    'Back':    (0., 0., 0., 0., 1.)
}

OH_to_label = {v:k for k,v in label_to_OH.items()}
idx_to_label = {np.argmax(k):v for k,v in OH_to_label.items()}

# C:\Users\itaym\Documents\limudim\semester_8\sequence_labler\output\Back_0000.csv
sample_files_dir = os.path.join('C:/','Users','itaym','Documents','limudim','semester_8','sequence_labler','output')

for sample_file in os.listdir(sample_files_dir):
    label, _ = sample_file.split('_')
    samples[label].append(pd.read_csv(os.path.join(sample_files_dir,sample_file)).infer_objects()[['accX','accY','accZ','gyroX','gyroY','gyroZ']].to_numpy())

# FIXME: remove hard coded labels here
X = tf.constant(samples['Rest'] + samples['Up'] + samples['Down'] + samples['Back'] + samples['Forward'])
y = tf.constant([label_to_OH['Rest'] for _ in samples['Rest']] +
                [label_to_OH['Up'] for _ in samples['Up']] +
                [label_to_OH['Down'] for _ in samples['Down']] +
                [label_to_OH['Back'] for _ in samples['Back']] +
                [label_to_OH['Forward'] for _ in samples['Forward']])

X_train, X_test, y_train, y_test = train_test_split(X.numpy(), y.numpy(), test_size = 0.2, random_state = 0)

## Data Preview

In [ ]:
for axes_idx, axes in enumerate(('Acceleration X',
                                 'Acceleration Y',
                                 'Acceleration Z',
                                 'Gyro X',
                                 'Gyro Y',
                                 'Gyro Z'
                                 )):
    figure = plt.figure(figsize=(10,10))
    figure.suptitle(f'{axes}')
    for dir_idx, dir in enumerate(samples.keys()):
        ax = figure.add_subplot(5,1,dir_idx+1)
        ax.set_title(dir)
        for sam in samples[dir]:
            ax.plot([i[axes_idx] for i in sam])

## Define Model

In [ ]:
model = MotionClassifier()

model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics = ['accuracy'])

## Training

In [ ]:
history = model.fit(
    X_train, y_train, epochs=N_EPOCHS, validation_data=(X_test, y_test), verbose=1
)

## Visualize Results

In [ ]:
epoch_range = range(1, N_EPOCHS+1)

figure = plt.figure(figsize=(10,10))
ax1 = figure.add_subplot(2,1,1)
ax1.set_title('Model accuracy')
ax1.plot(epoch_range, history.history['accuracy'])
ax1.plot(epoch_range, history.history['val_accuracy'])
ax1.set_title('Model accuracy')
ax1.set_ylabel('Accuracy')
ax1.set_xlabel('Epoch')
ax1.legend(['Train', 'Val'], loc='upper left')

# Plot training & validation loss values
ax2 = figure.add_subplot(2,1,2)
ax2.plot(epoch_range, history.history['loss'])
ax2.plot(epoch_range, history.history['val_loss'])
ax2.set_title('Model loss')
ax2.set_ylabel('Loss')
ax2.set_xlabel('Epoch')
ax2.legend(['Train', 'Val'], loc='upper left')

In [ ]:
y_true      = []
y_test_pred = []
# there got to be a better way to do this 😞
for i in np.array(y_test):
  y_true.append(OH_to_label[tuple(i)])

for i in np.array(model.predict(X_test)):
  y_test_pred.append(idx_to_label[np.argmax(i)])
  
ConfusionMatrixDisplay.from_predictions(y_true, y_test_pred, labels=list(label_to_OH.keys()))

## Saving Trained Model

In [ ]:
# model.save('my_model', include_optimizer=False)
model.save_weights('model_wheights')

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('my_model')
# converter._experimental_lower_tensor_list_ops = False
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False
tflite_model = converter.convert()

In [ ]:
model.summary()
for l in model.layers:
    print(l.name)
    for k in l.weights:
        print(f"\t{k.name} {k.shape}")

In [ ]:
print(model.call(tf.constant(np.random.randn(1,400,6))))